In [4]:
import numpy as np

# Sigmoid activation function (binary)
def sigmoid(yin):
    return 1 / (1 + np.exp(-yin))

# Bipolar sigmoid activation function
def bipolar_sigmoid(yin):
    return 2 / (1 + np.exp(-yin)) - 1

# Step function with two thresholds
def step_function(yin, lower_threshold, upper_threshold, input_type):
    if input_type == "binary":
        if yin < lower_threshold:
            return 0
        elif lower_threshold <= yin < upper_threshold:
            return 1
        else:
            return 1.5
    elif input_type == "bipolar":
        if yin < lower_threshold:
            return -1
        elif lower_threshold <= yin < upper_threshold:
            return 0
        else:
            return 1
    else:
        raise ValueError("Unknown input type. Must be 'binary' or 'bipolar'.")

# Threshold activation function
def threshold_function(yin, threshold):
    return 1 if yin >= threshold else 0

# Activation function based on input type and activation type
def activation_function(yin, lower_threshold, upper_threshold, threshold, activation_type, input_type):
    if activation_type == "thresh":
        return threshold_function(yin, threshold)
    elif activation_type == "step":
        return step_function(yin, lower_threshold, upper_threshold, input_type)
    elif activation_type == "sigmoid":
        if input_type == "bipolar":
            return bipolar_sigmoid(yin)
        else:
            return sigmoid(yin)
    else:
        raise ValueError("Unknown activation function type.")

# Perceptron learning algorithm with convergence check and delta weights
# Perceptron learning algorithm with convergence check and delta weights
def perceptron_learning(X, d, w1, w2, b, alpha, lower_threshold, upper_threshold, threshold, epochs, input_type, activation_type):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}:")
        print(f"{'x1':<3} {'x2':<3} {'Target':<7} {'yin':<7} {'Output y':<10} {'Δw1':<7} {'Δw2':<7} {'Δb':<5} {'w1':<7} {'w2':<7} {'b':<5}")

        weight_changed = False  # Track if weights are updated

        for i in range(len(X)):
            x1, x2 = X[i]  # Inputs
            target = d[i]  # Target output

            # Calculate weighted sum (yin)
            yin = w1 * x1 + w2 * x2 + b

            # Get output using the chosen activation function
            y = activation_function(yin, lower_threshold, upper_threshold, threshold, activation_type, input_type)

            # For sigmoid activation (binary), use threshold of 0.5
            if activation_type == "sigmoid" and input_type == "binary":
                y = 1 if y >= 0.5 else 0

            # For bipolar sigmoid, threshold should ideally be 0
            if activation_type == "sigmoid" and input_type == "bipolar":
                y = 1 if y >= 0 else -1

            # Compare y and target, only update if they are not equal
            if y != target:
                # Calculate weight and bias updates (delta values)
                delta_w1 = alpha * (target ) * x1
                delta_w2 = alpha * (target ) * x2
                delta_b = alpha * (target)

                # Update weights and bias
                w1 += delta_w1
                w2 += delta_w2
                b += delta_b

                # Mark that weights have changed
                weight_changed = True

                # Print detailed output for each iteration including delta weights
                print(f"{x1:<3} {x2:<3} {target:<7} {yin:<7.2f} {y:<10} {delta_w1:<7.2f} {delta_w2:<7.2f} {delta_b:<5.2f} {w1:<7.2f} {w2:<7.2f} {b:<5.2f}")
            else:
                # If y == target, there is no change, so print without deltas
                print(f"{x1:<3} {x2:<3} {target:<7} {yin:<7.2f} {y:<10} {'No Change':<7} {'No Change':<7} {'No Change':<5} {w1:<7.2f} {w2:<7.2f} {b:<5.2f}")

        # If no weights were changed during the epoch, the model has converged
        if not weight_changed:
            print("\nConverged! No further updates needed.")
            break

    return w1, w2, b

# Function to get user inputs for training data
def get_training_data(input_type):
    num_samples = int(input("Enter number of training samples: "))
    X = []
    d = []

    for i in range(num_samples):
        x1 = int(input(f"Enter x1 for sample {i+1}: "))
        x2 = int(input(f"Enter x2 for sample {i+1}: "))
        target = int(input(f"Enter target output for sample {i+1}: "))
        
        if input_type == "bipolar":
            x1 = -1 if x1 == 0 else 1
            x2 = -1 if x2 == 0 else 1
            target = -1 if target == 0 else 1

        X.append([x1, x2])
        d.append(target)

    return np.array(X), np.array(d)

# Function to get input type from user (binary or bipolar)
def get_input_type():
    while True:
        input_type = input("Enter input type (binary or bipolar): ").strip().lower()
        if input_type in ["binary", "bipolar"]:
            return input_type
        else:
            print("Invalid input type. Please enter 'binary' or 'bipolar'.")

# Function to get activation function from user (thresh, step or sigmoid)
def get_activation_function():
    while True:
        activation_type = input("Enter activation function (thresh, step or sigmoid): ").strip().lower()
        if activation_type in ["thresh", "step", "sigmoid"]:
            return activation_type
        else:
            print("Invalid activation function. Please enter 'thresh', 'step', or 'sigmoid'.")

# Function to get parameters based on the chosen activation function
def get_activation_parameters(activation_type):
    if activation_type == "thresh":
        threshold = float(input("Enter threshold value: "))
        return None, None, threshold
    elif activation_type == "step":
        lower_threshold = float(input("Enter lower threshold value (for step function): "))
        upper_threshold = float(input("Enter upper threshold value (for step function): "))
        return lower_threshold, upper_threshold, None
    elif activation_type == "sigmoid":
        threshold = float(input("Enter threshold value (for sigmoid function): "))
        return None, None, threshold

# Get parameters from the user
w1 = float(input("Enter initial weight w1: "))
w2 = float(input("Enter initial weight w2: "))
b = float(input("Enter initial bias: "))
alpha = float(input("Enter learning rate (alpha): "))
epochs = int(input("Enter the number of epochs: "))

# Get input type and activation function from the user
input_type = get_input_type()
activation_type = get_activation_function()

# Get the required parameters for the selected activation function
lower_threshold, upper_threshold, threshold = get_activation_parameters(activation_type)

# Get training data from user
X, d = get_training_data(input_type)

# Train the perceptron
final_w1, final_w2, final_b = perceptron_learning(X, d, w1, w2, b, alpha, lower_threshold, upper_threshold, threshold, epochs, input_type, activation_type)

# Print final results
print("\nFinal Weights and Bias after training:")
print(f"w1: {final_w1:.2f}, w2: {final_w2:.2f}, b: {final_b:.2f}")


Enter initial weight w1:  0
Enter initial weight w2:  0
Enter initial bias:  0
Enter learning rate (alpha):  1
Enter the number of epochs:  3
Enter input type (binary or bipolar):  binary
Enter activation function (thresh, step or sigmoid):  thresh
Enter threshold value:  0.5
Enter number of training samples:  4
Enter x1 for sample 1:  0
Enter x2 for sample 1:  0
Enter target output for sample 1:  0
Enter x1 for sample 2:  0
Enter x2 for sample 2:  1
Enter target output for sample 2:  1
Enter x1 for sample 3:  1
Enter x2 for sample 3:  0
Enter target output for sample 3:  1
Enter x1 for sample 4:  1
Enter x2 for sample 4:  1
Enter target output for sample 4:  1



Epoch 1:
x1  x2  Target  yin     Output y   Δw1     Δw2     Δb    w1      w2      b    
0   0   0       0.00    0          No Change No Change No Change 0.00    0.00    0.00 
0   1   1       0.00    0          0.00    1.00    1.00  0.00    1.00    1.00 
0   1   1       0.00    0          0.00    1.00    1.00  0.00    1.00    1.00 
1   0   1       1.00    1          No Change No Change No Change 0.00    1.00    1.00 
1   1   1       2.00    1          No Change No Change No Change 0.00    1.00    1.00 

Epoch 2:
x1  x2  Target  yin     Output y   Δw1     Δw2     Δb    w1      w2      b    
0   0   0       1.00    1          -0.00   -0.00   -1.00 0.00    1.00    0.00 
0   0   0       1.00    1          -0.00   -0.00   -1.00 0.00    1.00    0.00 
0   1   1       1.00    1          No Change No Change No Change 0.00    1.00    0.00 
1   0   1       0.00    0          1.00    0.00    1.00  1.00    1.00    1.00 
1   0   1       0.00    0          1.00    0.00    1.00  1.00    1.00    1.00 


In [ ]:
# Perceptron learning algorithm with convergence check and delta weights
def perceptron_learning(X, d, w1, w2, b, alpha, lower_threshold, upper_threshold, threshold, epochs, input_type, activation_type):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}:")
        print(f"{'x1':<3} {'x2':<3} {'Target':<7} {'yin':<7} {'Output y':<10} {'Δw1':<7} {'Δw2':<7} {'Δb':<5} {'w1':<7} {'w2':<7} {'b':<5}")

        weight_changed = False  # Track if weights are updated

        for i in range(len(X)):
            x1, x2 = X[i]  # Inputs
            target = d[i]  # Target output

            # Calculate weighted sum (yin)
            yin = w1 * x1 + w2 * x2 + b

            # Get output using the chosen activation function
            y = activation_function(yin, lower_threshold, upper_threshold, threshold, activation_type, input_type)

            # For sigmoid activation (binary), use threshold of 0.5
            if activation_type == "sigmoid" and input_type == "binary":
                y = 1 if y >= 0.5 else 0

            # For bipolar sigmoid, threshold should ideally be 0
            if activation_type == "sigmoid" and input_type == "bipolar":
                y = 1 if y >= 0 else -1

            # Compare y and target, only update if they are not equal
            if y != target:
                # Calculate weight and bias updates (delta values)
                delta_w1 = alpha * (target - y) * x1
                delta_w2 = alpha * (target - y) * x2
                delta_b = alpha * (target - y)

                # Update weights and bias
                w1 += delta_w1
                w2 += delta_w2
                b += delta_b

                # Mark that weights have changed
                weight_changed = True

                # Print detailed output for each iteration including delta weights
                print(f"{x1:<3} {x2:<3} {target:<7} {yin:<7.2f} {y:<10} {delta_w1:<7.2f} {delta_w2:<7.2f} {delta_b:<5.2f} {w1:<7.2f} {w2:<7.2f} {b:<5.2f}")
            else:
                # If y == target, there is no change, so print without deltas
                print(f"{x1:<3} {x2:<3} {target:<7} {yin:<7.2f} {y:<10} {'No Change':<7} {'No Change':<7} {'No Change':<5} {w1:<7.2f} {w2:<7.2f} {b:<5.2f}")

        # If no weights were changed during the epoch, the model has converged
        if not weight_changed:
            print("\nConverged! No further updates needed.")
            break

    return w1, w2, b


In [6]:
import numpy as np

# Sigmoid activation function (binary)
def sigmoid(yin):
    return 1 / (1 + np.exp(-yin))

# Bipolar sigmoid activation function
def bipolar_sigmoid(yin):
    return 2 / (1 + np.exp(-yin)) - 1

# Threshold activation function
def threshold_activation(yin, threshold):
    return 1 if yin >= threshold else 0

# Step function for binary and bipolar inputs
def step_function(yin, lower_threshold, upper_threshold, input_type):
    if input_type == "binary":
        if yin < lower_threshold:
            return 0
        elif lower_threshold <= yin < upper_threshold:
            return 1
        else:
            return 1.5  # For binary, we use 1.5 as the upper step
    elif input_type == "bipolar":
        if yin < lower_threshold:
            return -1
        elif lower_threshold <= yin < upper_threshold:
            return 0
        else:
            return 1
    else:
        raise ValueError("Unknown input type. Must be 'binary' or 'bipolar'.")

# Activation function based on user selection
def activation_function(yin, lower_threshold, upper_threshold, threshold, activation_type, input_type):
    if activation_type == "thresh":
        return threshold_activation(yin, threshold)
    elif activation_type == "step":
        return step_function(yin, lower_threshold, upper_threshold, input_type)
    elif activation_type == "sigmoid":
        if input_type == "bipolar":
            return bipolar_sigmoid(yin)
        else:
            return sigmoid(yin)
    else:
        raise ValueError("Unknown activation function type.")

# Perceptron learning algorithm with convergence check and delta weights
def perceptron_learning(X, d, w1, w2, b, alpha, lower_threshold, upper_threshold, threshold, epochs, input_type, activation_type):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}:")
        print(f"{'x1':<3} {'x2':<3} {'Target':<7} {'yin':<7} {'Output y':<10} {'Δw1':<7} {'Δw2':<7} {'Δb':<5} {'w1':<7} {'w2':<7} {'b':<5}")

        weight_changed = False  # Track if weights are updated

        for i in range(len(X)):
            x1, x2 = X[i]  # Inputs
            target = d[i]  # Target output

            # Calculate weighted sum (yin)
            yin = w1 * x1 + w2 * x2 + b

            # Get output using the chosen activation function
            y = activation_function(yin, lower_threshold, upper_threshold, threshold, activation_type, input_type)

            # For sigmoid activation (binary), use threshold of 0.5
            if activation_type == "sigmoid" and input_type == "binary":
                y = 1 if y >= 0.5 else 0

            # For bipolar sigmoid, threshold should ideally be 0
            if activation_type == "sigmoid" and input_type == "bipolar":
                y = 1 if y >= 0 else -1

            # Compare y and target, only update if they are not equal
            if y != target:
                # Calculate weight and bias updates (delta values)
                delta_w1 = alpha * (target ) * x1
                delta_w2 = alpha * (target ) * x2
                delta_b = alpha * (target )

                # Update weights and bias
                w1 += delta_w1
                w2 += delta_w2
                b += delta_b

                # Mark that weights have changed
                weight_changed = True

                # Print detailed output for each iteration including delta weights
                print(f"{x1:<3} {x2:<3} {target:<7} {yin:<7.2f} {y:<10} {delta_w1:<7.2f} {delta_w2:<7.2f} {delta_b:<5.2f} {w1:<7.2f} {w2:<7.2f} {b:<5.2f}")
            else:
                # If y == target, there is no change, so print without deltas
                print(f"{x1:<3} {x2:<3} {target:<7} {yin:<7.2f} {y:<10} {'No Change':<7} {'No Change':<7} {'No Change':<5} {w1:<7.2f} {w2:<7.2f} {b:<5.2f}")

        # If no weights were changed during the epoch, the model has converged
        if not weight_changed:
            print("\nConverged! No further updates needed.")
            break

    return w1, w2, b

# Example run (with hardcoded data to test, replace with user inputs if necessary)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # Example input
d = np.array([0, 1, 1, 1])  # Example target (for OR gate)

# Initial weights, learning rate, and parameters
w1, w2, b = 0.0, 0.0, 0.0
alpha = 1.0
lower_threshold = 0.5
upper_threshold = 1.5
threshold = 0.5
epochs = 10
input_type = "binary"
activation_type = "thresh"

# Run the perceptron learning algorithm
final_w1, final_w2, final_b = perceptron_learning(X, d, w1, w2, b, alpha, lower_threshold, upper_threshold, threshold, epochs, input_type, activation_type)

# Final weights and bias
print(f"\nFinal weights and bias: w1 = {final_w1:.2f}, w2 = {final_w2:.2f}, b = {final_b:.2f}")



Epoch 1:
x1  x2  Target  yin     Output y   Δw1     Δw2     Δb    w1      w2      b    
0   0   0       0.00    0          No Change No Change No Change 0.00    0.00    0.00 
0   1   1       0.00    0          0.00    1.00    1.00  0.00    1.00    1.00 
1   0   1       1.00    1          No Change No Change No Change 0.00    1.00    1.00 
1   1   1       2.00    1          No Change No Change No Change 0.00    1.00    1.00 

Epoch 2:
x1  x2  Target  yin     Output y   Δw1     Δw2     Δb    w1      w2      b    
0   0   0       1.00    1          0.00    0.00    0.00  0.00    1.00    1.00 
0   1   1       2.00    1          No Change No Change No Change 0.00    1.00    1.00 
1   0   1       1.00    1          No Change No Change No Change 0.00    1.00    1.00 
1   1   1       2.00    1          No Change No Change No Change 0.00    1.00    1.00 

Epoch 3:
x1  x2  Target  yin     Output y   Δw1     Δw2     Δb    w1      w2      b    
0   0   0       1.00    1          0.00    0.00    0.